# Tensors and Computation Graphs

[TOC]

TensorFlow uses a dataflow graph to represent your computation in terms of the dependencies between individual operations. This leads to a low-level programming model in which you first define the dataflow graph, then create a TensorFlow session to run parts of the graph across a set of local and remote devices.

## 1. How Does TensorFlow Work

You might think of TensorFlow Core programs as consisting of two discrete sections:
* Building the computational graph (tf.Graph).
* Running the computational graph (tf.Session).

### 1.1. Graph

A computation graph is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

* tf.Operation (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.

* tf.Tensor: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

C:\Users\Administrator\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0, dtype=tf.float32)
total = a + b
print(a)
print(b)
print(total)

Tensor("Const_12:0", shape=(), dtype=float32)
Tensor("Const_13:0", shape=(), dtype=float32)
Tensor("add_6:0", shape=(), dtype=float32)


**Note:** tf.Tensors do not have values, they are just handles to elements in the computation graph. 

We will go back to discuss how to build the computational graph later.

### 1.2. Session

To evaluate tensors, instantiate a tf.Session object, informally known as a session. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. * If a tf.Graph is like a .py file, a tf.Session is like the python executable.*

The following code creates a tf.Session object and then invokes its run method to evaluate the total tensor we created above:

In [4]:
sess = tf.Session()
print(sess.run(a))
print(sess.run(b))
print(sess.run(total))

3.0
4.0
7.0


### 1.3. Graph Visualization

TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.

First you save the computation graph to a TensorBoard summary file as follows:

In [5]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

This will produce an event file in the current directory with a name in the following format:
**events.out.tfevents.{timestamp}.{hostname}**

Now, in a new terminal, launch TensorBoard with the following shell command:
**tensorboard --logdir .**

Then open TensorBoard's [graphs page](http://localhost:6006/#graphs) in your browser, and you should see a graph.

For more about TensorBoard's graph visualization tools see [TensorBoard: Graph Visualization](https://www.tensorflow.org/guide/graph_viz).


## 2. Tensors

A Tensor is a symbolic **handle** to one of the outputs of an Operation. It does not hold the values of that operation's output, but instead provides a means of computing those values in a TensorFlow tf.Session.

This class has two primary purposes:

* A Tensor can be passed as an input to another Operation. This builds a dataflow connection between operations, which enables TensorFlow to execute an entire Graph that represents a large, multi-step computation. *That is, it is an edge in the computation graph.*

* After the graph has been launched in a session, the value of the Tensor can be computed by passing it to tf.Session.run. t.eval() is a shortcut for calling tf.get_default_session().run(t).

A tf.Tensor has the following properties:

* A data type (float32, int32, or string, for example). Each element in the Tensor has the same data type, and the data type is always known.

* A shape. The shape (that is, the number of dimensions it has and the size of each dimension) might be only partially known. Most operations produce tensors of fully-known shapes if the shapes of their inputs are also fully known, but in some cases it's only possible to find the shape of a tensor at graph execution time.

* An Operation. Operation that computes this tensor.

The main tensors include:
* tf.Variable
* tf.constant
* tf.placeholder
* tf.SparseTensor

With the exception of tf.Variable, the value of a tensor is immutable, which means that in the context of a single execution tensors only have a single value. However, evaluating the same tensor twice can return different values; for example that tensor can be the result of reading data from disk, or generating a random number.


In [14]:
c = tf.constant([1.0,2,3], dtype=tf.float32)
d = tf.Variable([4,5,6.0], dtype=tf.float32)
total = c + d

print(c)
print(d)
print(total)
print(total.op)

Tensor("Const_23:0", shape=(3,), dtype=float32)
<tf.Variable 'Variable_1:0' shape=(3,) dtype=float32_ref>
Tensor("add_18:0", shape=(3,), dtype=float32)
name: "add_18"
op: "Add"
input: "Const_23"
input: "Variable_1/read"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}



### 2.1 Transformation between Tensors and Numpy Arrays 

Once the computation graph has been built, you can run the computation that produces a particular tf.Tensor and fetch the value assigned to it. This is often useful for debugging as well as being required for much of TensorFlow to work.

* The simplest way to evaluate a Tensor is using the Tensor.eval method. For example:

In [23]:
constant = tf.constant([1, 2, 3])
tensor = constant * constant
sess = tf.Session()
## Transform a tensor into a numpy array
fetch_value = tensor.eval(session=sess)
print(fetch_value)
print(type(tensor))
print(type(fetch_value))

[1 4 9]
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'numpy.ndarray'>


* One can then use tf.convert_to_tensor to transform a numpy array into tensor. For example:

In [26]:
## Transform a numpy array into a tensor
trans_tensor = tf.convert_to_tensor(fetch_value)
print(trans_tensor)
print(type(trans_tensor))

Tensor("Const_35:0", shape=(3,), dtype=int32)
<class 'tensorflow.python.framework.ops.Tensor'>


## 3. Building a tf.Graph

Most TensorFlow programs start with a computation graph construction phase. In this phase, you invoke TensorFlow API functions that construct new tf.Operation (node) and tf.Tensor (edge) objects and add them to a tf.Graph instance. TensorFlow provides a default graph that is an implicit argument to all API functions in the same context. For example:

* Calling tf.constant(42.0) creates a single tf.Operation that produces the value 42.0, adds it to the default graph, and returns a tf.Tensor that represents the value of the constant.

* Calling tf.matmul(x, y) creates a single tf.Operation that multiplies the values of tf.Tensor objects x and y, adds it to the default graph, and returns a tf.Tensor that represents the result of the multiplication.

* Executing v = tf.Variable(0) adds to the graph a tf.Operation that will store a writeable tensor value that persists between tf.Session.run calls. The tf.Variable object wraps this operation, and can be used like a tensor, which will read the current value of the stored value. The tf.Variable object also has methods such as tf.Variable.assign and tf.Variable.assign_add that create tf.Operation objects that, when executed, update the stored value.

* Calling tf.train.Optimizer.minimize will add operations and tensors to the default graph that calculates gradients, and return a tf.Operation that, when run, will apply those gradients to a set of variables.


In the following, we present some examples to how to build the computation graph.

### 3.1. A simple example: computing a=(b+c)∗(c+2)

In [6]:
const = tf.constant(2.0, name='const')

b = tf.Variable(2.0, name='b')
c = tf.Variable(1.0, dtype=tf.float32, name='c')
d = tf.add(b, c, name='d')
e = tf.add(c, const, name='e')
a = tf.multiply(d, e, name='a')
print(a)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
a_out = sess.run(a)
print(a_out)

writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()



Tensor("a:0", shape=(), dtype=float32)
9.0


### 3.2. Matrix Multiplication

In [7]:
A = tf.constant([1,2,3],dtype=tf.float32)
B = tf.Variable(tf.zeros([1,3])) + A
C = tf.Variable([[1,2,3],[2,4,6],[3,6,9]],dtype=tf.float32)
D = tf.random_normal([1,3])

E = tf.matmul(B,C) + D

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
print("A is {}".format(sess.run(A)))
print("B is {}".format(sess.run(B)))
print("C is {}".format(sess.run(C)))
print("D is {}".format(sess.run(D)))
print("E is {}".format(sess.run(E)))

writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()



A is [1. 2. 3.]
B is [[1. 2. 3.]]
C is [[1. 2. 3.]
 [2. 4. 6.]
 [3. 6. 9.]]
D is [[-1.0206345  -1.9120213   0.45873082]]
E is [[13.323662 28.117954 45.0393  ]]


### 3.3. Placeholder

A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later, like a function argument.

In [8]:
input = tf.placeholder(tf.float32, [None, 1], name='input')
res = input + 0.1;

sess = tf.Session()
output = sess.run(res, feed_dict={input: np.arange(0, 10)[:, np.newaxis]})
print("Variable a is {}".format(output))

Variable a is [[0.1]
 [1.1]
 [2.1]
 [3.1]
 [4.1]
 [5.1]
 [6.1]
 [7.1]
 [8.1]
 [9.1]]


In [ ]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

### 3.4. A Neural Network Example**


In [9]:
# Load data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Define parameters
learning_rate = 0.5
epochs = 10
batch_size = 100

# Placeholder
# The size of the input image is 28 x 28 = 784
x = tf.placeholder(tf.float32, [None, 784])
# The output size is 0-9 one-hot label
y = tf.placeholder(tf.float32, [None, 10])

# hidden layer => w1, b1
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

# output layer => w, b
W2 = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')
y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))

# Define the loss function
y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y) * tf.log(1 - y_clipped), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

# Define the accuracy function
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Define init operator
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # Initialize the varaibles
    sess.run(init)
    total_batch = int(len(mnist.train.labels) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimizer, cross_entropy], feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost = ", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 1 cost =  0.742
Epoch: 2 cost =  0.263
Epoch: 3 cost =  0.194
Epoch: 4 cost =  0.159
Epoch: 5 cost =  0.134
Epoch: 6 cost =  0.114
Epoch: 7 cost =  0.100
Epoch: 8 cost =  0.085
Epoch: 9 cost =  0.074
Epoch: 10 cost =  0.067
0.975


Note: 

* tf.clip_by_value(A, min, max)：for the input tensor A，transform each element of A into the range between min and max.
* reduce_sum Compute the sum of elements across the indicated dimensions of the input tensor.
* reduce_mean Compute the mean of elements across the indicated dimensions of the input tensor.

In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

## 4. Graph Execution

### 4.1. Single-Device Execution

Let’s first consider the simplest execution scenario: a single worker process with a single device. The nodes of the graph are executed in an order that respects the dependencies between nodes (a topological ordering). In particular, we keep track of a count per node of the number of dependencies of that node that have not yet been executed. Once this count drops to zero, the node is eligible for execution and is added to a ready queue. The ready queue is processed in some unspecified order, delegating execution of the kernel for a node to the device object. When a node has finished executing, the counts of all nodes that depend on the completed node are decremented.

Note that the formed computation graph is a directed acyclic graph (DAG), and the in-degree of each node is the number of dependencies of that node that have not yet been executed. The Graph Execution algorithm can be discribed as follows:

1. Initiate an array to count the in-degree of each node in the computation graph. 

2. Initiate a queue, and push all nodes with zero in-degree into the queue.

3. While the queue is not empty, pop one node to excute, remove this node from the graph.

4. Decrease by 1 the in-degree of all nodes that depend on this excuted node. When those node's in-degrees become zero, push them into the queue.

5. Repeat Steps 3-4 until the queue become empty.


### 4.2. Multi-Device Execution

Once a system has multiple devices, there are two main complications: 
* Deciding which device to place the computation for each node in the graph;
* Managing the required communication of data across device boundaries implied by these placement decisions. 

One can refer to the White Paper of TensorFlwo for more detail about the Multi-Device Execution.